# Bibliotecas

In [2]:
import sys

sys.path.append('/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/')


import json
from pathlib import Path

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import hepherolib.analysis as ana

from lgbm.controllers_lgbm import LGBLearner, LGBModel

ana.start()
plt.style.use("default")

ModuleNotFoundError: No module named 'hepherolib'

# Configuração

In [2]:
period = 'APV_16'
year_style = 2016
dataset_year = "APV_2016"
basedir = '/eos/user/t/thenriqu/Dark_Matter/Amostras/hhdmAnalysis_deepJet_Regions/datasets'
dataset_name = basedir.split('/')[-2]

# Data folder
data_path = f"./data/{dataset_name}/{dataset_year}"

# Setup output folders
models_path = f"./models/{dataset_name}/{dataset_year}"
Path(models_path).mkdir(parents=True, exist_ok=True)

# Lendo os dados

In [3]:
model_name = "LGB_multi_signal"

X_train = pd.read_csv(f"{data_path}/{model_name}-train-data.csv")
X_test = pd.read_csv(f"{data_path}/{model_name}-test-data.csv")
W_train = X_train.pop("modelWeight").values
W_test = X_test.pop("modelWeight").values
Y_train = X_train.pop("Label").values
Y_test = X_test.pop("Label").values
evtWeight_train = X_train.pop("evtWeight").values
evtWeight_test = X_test.pop("evtWeight").values

In [4]:
X_train.head()

,LeadingLep_pt,LepLep_deltaM,LepLep_deltaR,LepLep_pt,MET_LepLep_Mt,MET_LepLep_deltaPhi,MET_pt,MT2LL,Nbjets,TrailingLep_pt
0,127.877914,5.665382,1.004920,184.44044,197.350770,2.764841,54.709740,96.820950,1,77.357370
1,113.009460,3.599693,1.009094,178.61588,147.659620,1.591687,59.785347,63.220463,1,79.000790
2,97.949240,22.735489,2.811170,67.28066,119.337814,3.041180,53.051918,14.674115,1,33.556896
3,311.434630,4.510574,0.451174,423.52713,882.859400,3.096336,460.324680,393.019100,1,120.251110
4,144.094960,24.149925,2.172507,130.71048,313.220800,2.877176,190.960710,61.069447,2,28.793385


In [5]:
W_train

array([1.24207517e+00, 7.10710006e-04, 1.56460205e+00, ...,
       2.76548358e-01, 1.13629105e-03, 1.06929979e-01])

In [6]:
Y_train

array([0, 0, 0, ..., 0, 0, 0])

# Procura por Hiperparametros

#### Usado pelo Macedo - LGBMClassifier

n_iter = 150
cv = 2
param_grid = {
    'learning_rate': [0.008, 0.0004],
    'n_estimators': [900, 1000],
    'num_leaves': [24,128,62],
    'min_child_samples':[100,400,800],
    'subsample':[1,0.9, 0.8],
    'colsample_bytree':[0.9,0.8,0.2],
    'objective': ['multiclass'],
    'metric': ['multi_logloss'],
    'num_class':[3],
    'boosting_typ’: [‘gbdt’]
    'feature_pre_filter': ['False']
}




#### Usado pelo Gabriel: XGBoostClassifier


params = {
    'n_estimators': [100, 500, 1000, 2000, 3000, 5000, 7000, 9000],
    'min_child_weight': range(1,10),
    'learning_rate': [1e-3, 1e-2, 1e-1, 0.3, 0.5, 0.7, 0.9, 1.],
    'subsample': np.arange(0.50, 1.01, 0.05),
    'colsample_bytree': np.arange(0.50, 1.01, 0.05),
    'max_depth': range(3,11),
    'eval_metric': ['logloss', 'error']
}






#### Usado pelo Thiago ( Eu ): vottingclassifier 


log_clf = LogisticRegression(solver = 'newton-cg', random_state = 42)
rnd_clf = XGBClassifier( objective = 'multiclass', n_estimators = 4000, learning_rate = 0.001, random_state = 42)
lgbm_clf = LGBMClassifier(objective = 'multiclass',  num_class = 3, n_estimators = 4000, metric = 'multi_logloss', learning_rate = 0.001, boosting_type = 'gbdt', random_state = 42)

voting_clf_soft = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('lgbm', lgbm_clf)],
    voting='soft')



In [7]:

params = {
    'n_estimators': [900, 1000, 5000, 7000, 9000],
    'min_child_weight': range(1,10),
    'min_child_samples': [100, 400, 800],
    'learning_rate': [0.001, 0.004, 0.008],
    'subsample': [1, 0.9, 0.8],
    'colsample_bytree': [0.9, 0.8, 0.2],
#     'colsample_bytree': np.arange(0.50, 1.01, 0.05),
#     'subsample': np.arange(0.50, 1.01, 0.05),
    'max_depth': range(3,11),
    'eval_metric': ['logloss', 'error']
}



learner = LGBLearner(X_train, Y_train, W_train, njobs=12)
res = learner.find_hyperparams(hyperparams_grid=params, n_splits=5, n_iter=30)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


/cvmfs/sft.cern.ch/lcg/views/LCG_105a_swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 54070, number of negative: 519200
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2302
[LightGBM] [Info] Number of data points in the train set: 573270, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666695 -> initscore=0.693273
[LightGBM] [Inf

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 54071, number of negative: 519200
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003366 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 573271, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666480 -> initscore=0.692308
[LightGBM] [Inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.115507 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 573271, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666480 -> initscore=0.692308
[LightGBM] [Info] Start training from score 0.692308
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 67588, number of negative: 649000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007377 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2303
[LightGBM] [Info] Number of data points in the train set: 716588, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.666667 -> initscore=0.693147
[LightGBM] [Inf

# Salvando os Parametros

In [8]:
with open(f'{models_path}/{model_name}-bst-params.json', 'w') as f:
    json.dump(res['hyperparameters'], f, ensure_ascii=False, indent=4)

# Noficação Quando o Notebook terminar de compilar

In [9]:
from IPython.display import clear_output, display, HTML, Javascript

display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "Process completed!";
  window.speechSynthesis.speak(msg);
  alert("Process completed!")
"""))

<IPython.core.display.Javascript object>